In [ ]:
import numpy as np
import mindspore
import mindspore.nn as nn

from mindspore import Parameter, Tensor

In [ ]:
def make_batch(sentences, word_dict):
    input_batch = []
    target_batch = []
    
    for sent in sentences:
        word = sent.split()
        inp = [word_dict[n] for n in word[:-1]]
        tgt = word_dict[word[-1]]
        
        input_batch.append(inp)
        target_batch.append(tgt)
    return input_batch, target_batch

In [ ]:
class NNLM(nn.Cell):
    def __init__(self, n_steps, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.C = nn.Embedding(vocab_size, embed_size)
        self.H = nn.Dense(n_steps * embed_size, hidden_size, has_bias=False)
        self.d = Parameter(Tensor(np.ones(hidden_size), mindspore.float32))
        self.U = nn.Dense(hidden_size, vocab_size, has_bias=False)
        self.W = nn.Dense(n_steps * embed_size, vocab_size, has_bias=False)
        self.b = Parameter(Tensor(np.ones(vocab_size), mindspore.float32))
        self.n_steps = n_steps
        self.embed_size = embed_size
        self.tanh = nn.Tanh()
    def construct(self, X):
        X = self.C(X)
        X = X.view(-1, self.n_steps * self.embed_size)
        tanh = self.tanh(self.d + self.H(X))
        output = self.b + self.W(X) + self.U(tanh)
        return output
        

In [ ]:
n_steps = 2
hidden_size = 2
embed_size = 2

sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

In [ ]:
input_batch, target_batch = make_batch(sentences, word_dict)
input_batch = Tensor(input_batch, mindspore.int32)
target_batch = Tensor(target_batch)
target_batch

In [ ]:
model = NNLM(n_steps, vocab_size, embed_size, hidden_size)

In [ ]:
criterion = nn.SoftmaxCrossEntropyWithLogits(sparse=True,reduction='mean')
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

In [ ]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

net_with_criterion = nn.WithLossCell(model, criterion)
train_network = nn.TrainOneStepCell(net_with_criterion, optimizer)
train_network.set_train()
epoch = 5000
for step in range(epoch):
    loss = train_network(input_batch, target_batch)
    if (step + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (step + 1), 'cost = ', '{:.6f}'.format(loss.asnumpy()))

In [ ]:
train_network.set_train(False)
predict = model(input_batch).asnumpy().argmax(axis=1)
print(predict)
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict])